<a href="https://colab.research.google.com/github/Ellinei/229352-StatisticalLearning/blob/main/Lab04_Naive_Bayes_Grid_and_Random_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #4

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from scipy.stats import uniform

In [ ]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

### Naive Bayes [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)

In [ ]:
n = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])

n.fit(Xtrain, ytrain)

### Random Search Cross-Validation [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

### Uniform distribution in `Scipy` [(Documentation)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html)

In [ ]:
uniform_dist = uniform(loc=0, scale=1)

samples = uniform_dist.rvs(size=10, random_state=42)
print("Random samples from uniform distribution:", samples)

#### Exercise

1. For the Naive Bayes model, use grid search 5-fold cross-validation across different values of `alpha` to find the best model.

2. For the best value of `alpha`, compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* What is the model's `f1_macro` score?

3. Repeat Exercise 1 and 2 for **random search** 5-fold cross validation across different values of `alpha`. Compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* Did you get a better `f1_macro` score compared to grid search in Exercise 2?

Grid Search Cross-Validation

In [ ]:
param_grid = {'clf__alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]}

grid_search = GridSearchCV(n, param_grid, cv=5, scoring='f1_macro')

grid_search.fit(Xtrain, ytrain)

print("Best parameters (Grid Search):", grid_search.best_params_)
print("Best f1_macro score (Grid Search):", grid_search.best_score_)

Evaluate Grid Search Best Model on Test Set

In [ ]:
best_grid_model = grid_search.best_estimator_

y_pred_grid = best_grid_model.predict(Xtest)

f1_macro_grid = classification_report(ytest, y_pred_grid, output_dict=True)['macro avg']['f1-score']
print("f1_macro score on test set (Grid Search):", f1_macro_grid)

print("Best alpha value (Grid Search):", grid_search.best_params_['clf__alpha'])

Random Search Cross-Validation and Evaluation

In [ ]:
param_dist = {'clf__alpha': uniform(loc=0, scale=1)}

random_search = RandomizedSearchCV(n, param_dist, n_iter=10, cv=5, scoring='f1_macro', random_state=42)

random_search.fit(Xtrain, ytrain)

print("Best parameters (Random Search):", random_search.best_params_)
print("Best f1_macro score (Random Search):", random_search.best_score_)

best_random_model = random_search.best_estimator_

y_pred_random = best_random_model.predict(Xtest)

f1_macro_random = classification_report(ytest, y_pred_random, output_dict=True)['macro avg']['f1-score']
print("f1_macro score on test set (Random Search):", f1_macro_random)

print("Best alpha value (Random Search):", random_search.best_params_['clf__alpha'])

print("\nComparison:")
print(f"Grid Search f1_macro on test set: {f1_macro_grid}")
print(f"Random Search f1_macro on test set: {f1_macro_random}")